<a href="https://colab.research.google.com/github/gershomrichardbruno/Ergonomics-Smart-Chair/blob/main/LeftSide_Pose_HipKnee_Auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Left-Side Pose Analyzer (Neck–Hip–Knee–Ankle) — AUTO

What it does:
- **Left side only**
- Per-frame **MediaPipe Pose** detection (no optical-flow drift)
- Markers: **Neck** (midpoint of left shoulder & left ear), **Hip**, **Knee**, **Ankle**
- Angles: **Hip** = Shoulder–Hip–Knee; **Knee** = Hip–Knee–Ankle
- Smoothing with a simple moving average
- Exports: CSV + annotated MP4 and auto-downloads them

How to use:
1. Run **Setup**
2. Run **Upload left-side video**
3. Run **Analyze & Export**


In [1]:
# === Setup (no SciPy to avoid version conflicts) ===
!pip -q install "opencv-python-headless>=4.9.0.80,<4.13" "mediapipe==0.10.14" "matplotlib>=3.8,<3.10"

import cv2, numpy as np, os, csv, warnings
import mediapipe as mp
import matplotlib.pyplot as plt
from google.colab import files

warnings.filterwarnings('ignore')

def moving_average(y, window=7):
    y = np.asarray(y, float)
    if window < 3 or len(y) < window:
        return y
    yy = y.copy()
    mask = np.isnan(yy)
    if np.any(mask):
        idx = np.where(~mask, np.arange(len(yy)), 0)
        np.maximum.accumulate(idx, out=idx)
        yy = yy[idx]
    kernel = np.ones(window) / window
    pad = window // 2
    yy_pad = np.pad(yy, (pad, pad), mode='edge')
    return np.convolve(yy_pad, kernel, mode='valid')

def angle_three_points(a, b, c):
    ba = np.array([a[0]-b[0], a[1]-b[1]], dtype=float)
    bc = np.array([c[0]-b[0], c[1]-b[1]], dtype=float)
    if np.linalg.norm(ba) < 1e-6 or np.linalg.norm(bc) < 1e-6:
        return np.nan
    cosang = np.dot(ba, bc) / (np.linalg.norm(ba)*np.linalg.norm(bc))
    cosang = np.clip(cosang, -1.0, 1.0)
    return np.degrees(np.arccos(cosang))

print('✅ Setup complete')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
✅ Setup complete


## Upload left-side video

In [2]:
print('Upload your left-side video (MP4/MOV/etc.)')
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print('Using:', video_path)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError('Error opening video file. Check path or upload again.')
fps = cap.get(cv2.CAP_PROP_FPS) or 24.0
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
ret, first_frame = cap.read(); cap.release()
if not ret:
    raise IOError("Couldn't read first frame.")
print(f'FPS: {fps:.2f}, Frames: {n_frames}')


Upload your left-side video (MP4/MOV/etc.)


Saving Rakshan left (2).mov to Rakshan left (2).mov
Using: Rakshan left (2).mov
FPS: 29.97, Frames: 9235


## Analyze & Export

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.6,
                    min_tracking_confidence=0.6)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS) or 24.0
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

neck_xy = np.full((n_frames, 2), np.nan, float)
hip_xy  = np.full((n_frames, 2), np.nan, float)
knee_xy = np.full((n_frames, 2), np.nan, float)
ank_xy  = np.full((n_frames, 2), np.nan, float)
sh_xy   = np.full((n_frames, 2), np.nan, float)  # left shoulder (internal)

vis_th = 0.6
last = { 'neck': None, 'hip': None, 'knee': None, 'ank': None, 'sh': None }
jump_px = max(20, int(0.06 * max(w,h)))

def accept(prev, new):
    if prev is None: return new
    d = np.linalg.norm(np.array(prev)-np.array(new))
    if d > jump_px:
        return 0.7*np.array(prev) + 0.3*np.array(new)
    return new

i = 0
while True:
    ret, frame = cap.read()
    if not ret: break
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = pose.process(rgb)
    if res.pose_landmarks:
        lms = res.pose_landmarks.landmark
        def get_xy(lm):
            return np.array([lm.x * w, lm.y * h], float)

        ls = lms[mp_pose.PoseLandmark.LEFT_SHOULDER]
        lh = lms[mp_pose.PoseLandmark.LEFT_HIP]
        lk = lms[mp_pose.PoseLandmark.LEFT_KNEE]
        la = lms[mp_pose.PoseLandmark.LEFT_ANKLE]
        le = lms[mp_pose.PoseLandmark.LEFT_EAR]

        if ls.visibility>vis_th: sh = get_xy(ls); last['sh']=sh
        else: sh = last['sh']

        if lh.visibility>vis_th: hip = get_xy(lh); hip = accept(last['hip'], hip); last['hip']=hip
        else: hip = last['hip']

        if lk.visibility>vis_th: knee = get_xy(lk); knee = accept(last['knee'], knee); last['knee']=knee
        else: knee = last['knee']

        if la.visibility>vis_th: ank = get_xy(la); ank = accept(last['ank'], ank); last['ank']=ank
        else: ank = last['ank']

        if (ls.visibility>vis_th) and (le.visibility>vis_th):
            ear = get_xy(le)
            neck = 0.5*(get_xy(ls)+ear)
            neck = accept(last['neck'], neck); last['neck']=neck
        else:
            neck = last['neck']
    else:
        sh = last['sh']; hip = last['hip']; knee = last['knee']; ank = last['ank']; neck = last['neck']

    if neck is not None: neck_xy[i]=neck
    if hip  is not None: hip_xy[i]=hip
    if knee is not None: knee_xy[i]=knee
    if ank  is not None: ank_xy[i]=ank
    if sh   is not None: sh_xy[i]=sh
    i += 1

cap.release(); pose.close()
n_frames = i
t = np.arange(n_frames) / fps

for arr in (neck_xy, hip_xy, knee_xy, ank_xy, sh_xy):
    arr[:n_frames,0] = moving_average(arr[:n_frames,0], window=7)
    arr[:n_frames,1] = moving_average(arr[:n_frames,1], window=7)

hip_angle  = np.full(n_frames, np.nan)
knee_angle = np.full(n_frames, np.nan)
for k in range(n_frames):
    S = tuple(sh_xy[k]) if np.all(np.isfinite(sh_xy[k])) else None
    H = tuple(hip_xy[k]) if np.all(np.isfinite(hip_xy[k])) else None
    K = tuple(knee_xy[k]) if np.all(np.isfinite(knee_xy[k])) else None
    A = tuple(ank_xy[k]) if np.all(np.isfinite(ank_xy[k])) else None
    if S and H and K:
        hip_angle[k] = angle_three_points(S, H, K)
    if H and K and A:
        knee_angle[k] = angle_three_points(H, K, A)

os.makedirs('outputs', exist_ok=True)
csv_path = 'outputs/posture_left_side_timeseries.csv'
with open(csv_path, 'w', newline='') as f:
    wtr = csv.writer(f)
    header = ['frame','time_s',
              'Neck_x','Neck_y','Hip_x','Hip_y','Knee_x','Knee_y','Ankle_x','Ankle_y',
              'hip_angle_deg','knee_angle_deg']
    wtr.writerow(header)
    for k in range(n_frames):
        row = [k, float(t[k])]
        row += [float(neck_xy[k,0]), float(neck_xy[k,1])]
        row += [float(hip_xy[k,0]),  float(hip_xy[k,1])]
        row += [float(knee_xy[k,0]), float(knee_xy[k,1])]
        row += [float(ank_xy[k,0]),  float(ank_xy[k,1])]
        row += [float(hip_angle[k]), float(knee_angle[k])]
        wtr.writerow(row)
print('Saved CSV:', csv_path)

cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_path = 'outputs/posture_left_side_annotated.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(out_path, fourcc, fps if fps>0 else 24.0, (w, h))
font = cv2.FONT_HERSHEY_SIMPLEX

for k in range(n_frames):
    ret, frame = cap.read()
    if not ret: break
    def draw_point(pt, name):
        if np.all(np.isfinite(pt)):
            x,y = int(pt[0]), int(pt[1])
            cv2.circle(frame, (x,y), 6, (0,255,0), -1)
            cv2.putText(frame, name, (x+8, y-8), font, 0.6, (255,255,255), 2, cv2.LINE_AA)

    def line_if(a,b,color=(255,200,0)):
        if np.all(np.isfinite(a)) and np.all(np.isfinite(b)):
            cv2.line(frame, tuple(a.astype(int)), tuple(b.astype(int)), color, 3)

    draw_point(neck_xy[k], 'Neck')
    draw_point(hip_xy[k],  'Hip')
    draw_point(knee_xy[k], 'Knee')
    draw_point(ank_xy[k],  'Ankle')
    line_if(sh_xy[k], hip_xy[k])
    line_if(hip_xy[k], knee_xy[k])
    line_if(knee_xy[k], ank_xy[k])

    y0 = 30; dy = 28
    lines = []
    if np.isfinite(hip_angle[k]):  lines.append(f'Hip angle: {hip_angle[k]:.1f} deg')
    if np.isfinite(knee_angle[k]): lines.append(f'Knee angle: {knee_angle[k]:.1f} deg')
    for j, txt in enumerate(lines):
        cv2.putText(frame, txt, (20, y0 + j*dy), font, 0.8, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(frame, txt, (20, y0 + j*dy), font, 0.8, (255,255,255), 1, cv2.LINE_AA)

    out.write(frame)

cap.release(); out.release()
print('Saved annotated video:', out_path)

files.download(csv_path)
files.download(out_path)
print('✅ Done and downloaded')


Saved CSV: outputs/posture_left_side_timeseries.csv
Saved annotated video: outputs/posture_left_side_annotated.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done and downloaded
